In [2]:
import praw
import datetime as dt
import pandas as pd
reddit = praw.Reddit(client_id='Rlmvoi5snEil5RLoeCymJw',
                        client_secret='oMmJOOfBMoCjtROy4WDOikhvCxF-zw',
                        user_agent='foosh')
headlines = set()

def get_reddit_post(stock_ticker):
    subreddit_list = ['stocks','wallstreetbets','stockmarket','options']    

    list_of_columns = ['stock_ticker','subreddit','id','title','url','upvotes','comments','author','created_time']
    df = pd.DataFrame(columns = list_of_columns)
    stock_ticker_lists = []
    subreddit_lists = []
    id = []
    title = []
    url = []
    upvotes = []
    comments = []
    author = []
    created_time = []

    for subreddit in subreddit_list:
        praw = reddit.subreddit(subreddit)
        posts = praw.search(stock_ticker, sort='all', time_filter='hour')

        for submission in posts:
            subreddit_lists.append(subreddit)
            stock_ticker_lists.append(stock_ticker)
            url.append(submission.url)
            title.append(submission.title)
            upvotes.append(submission.score)
            comments.append(submission.num_comments)
            author.append(submission.author)
            id.append(submission.id)
            created_time.append(submission.created_utc)
        
    df['stock_ticker'] = stock_ticker_lists
    df['subreddit'] = subreddit_lists
    df['id'] = id
    df['title'] = title
    df['url'] = url
    df['upvotes'] = upvotes
    df['comments'] = comments
    df['author'] = author
    df['created_time'] = created_time
    
    print(len(df))
    return df


Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [3]:
# top 30 US tech firms by market cap

def get_all_tickers():
    ticker_list = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NVDA", "META", "AVGO", "ORCL", "CSCO", 
    "CRM", "TXN", "ADBE", "NFLX", "QCOM", "AMD", "IBM", "INTU", "INTC", "AMAT",
    "BKNG", "ADI", "ADP", "ServiceNow", "PYPL", "ABNB", "FISV", "LRCX", "UBER", "EQIX"]

    output = pd.DataFrame()
    for ticker in ticker_list:
        df = get_reddit_post(ticker)
        output = pd.concat([output,df])
    return output



In [4]:
def get_id_ticker(df):
    df['stock_ticker'] = df['stock_ticker'].replace('ServiceNow','NOW')
    id_ticker_dict = {}
    # id_fields_dict = {}
    
    for index, row in df.iterrows():
        id = row['id']
        ticker = row['stock_ticker']
        # fields = [row['subreddit'], row['title'], row['url'],row['upvotes'],row['comments'],row['author'],row['created_time']]
        if id in id_ticker_dict:
            value = id_ticker_dict[id]
            value.append(ticker)
            id_ticker_dict[id] = value
        else:
            id_ticker_dict[id] = [ticker]
        # id_fields_dict[id] = fields
        
    # df = pd.DataFrame(list(id_ticker_dict.items()),columns = ['id','ticker']) 
    # df.sort_values('id')

    return id_ticker_dict





In [5]:
def get_full_table(df):
    id_ticker_dict = get_id_ticker(df)
    df.drop(['stock_ticker'], axis=1)
    df.drop_duplicates()
    tickers = []
    for index, row in df.iterrows():
        tickers.append(id_ticker_dict[row['id']])
    df['tickers'] = tickers
    return df

def get_table_with_duplicates(df):
    df = df.drop(['subreddit','title','url','comments','upvotes','author','created_time'], axis=1)
    return df


In [6]:
df = get_all_tickers()
output = get_full_table(df)
output2 = get_table_with_duplicates(df)

KeyboardInterrupt: 

In [4]:
from big_query import *
df = pd.read_csv('reddit.csv')
load_dataframe_to_bigquery(df, "Reddit.fulltable")



100%|██████████| 1/1 [00:00<?, ?it/s]


In [ ]:
df

,Unnamed: 0,stock_ticker,subreddit,id,title,url,upvotes,comments,author,created_time
0,0,AAPL,stocks,11tm5qq,r/Stocks Daily Discussion & Fundamentals Frida...,https://www.reddit.com/r/stocks/comments/11tm5...,44,534,AutoModerator,1.679045e+09
1,1,AAPL,stocks,11o76ht,What stocks are you eyeing if the market goes ...,https://www.reddit.com/r/stocks/comments/11o76...,127,279,Zabazooo,1.678498e+09
2,2,AAPL,stocks,11t3eke,safe haven The stock market is red-eyed. At pr...,https://www.reddit.com/r/stocks/comments/11t3e...,6,11,ContentFinding58,1.678994e+09
3,3,AAPL,stocks,11qdthr,Keep portfolio / Sell portfolio and put all in...,https://www.reddit.com/r/stocks/comments/11qdt...,5,6,Drragos,1.678724e+09
4,4,AAPL,wallstreetbets,11o5jer,How major US banks could be at risk of sweepin...,https://www.reddit.com/r/wallstreetbets/commen...,75,73,zulufux999,1.678494e+09
5,5,AAPL,wallstreetbets,11td8p7,🚀 $GOOGL: YOLOing on AI Beasts - BARD & OpenAI...,https://www.reddit.com/r/wallstreetbets/commen...,6,11,browndroid,1.679017e+09
6,6,AAPL,wallstreetbets,11rd2i1,Top changes to open interest calls and puts (T...,https://www.reddit.com/r/wallstreetbets/commen...,7,7,628rand,1.678816e+09
7,7,AAPL,wallstreetbets,11s9rhp,Top changes to open interest calls and puts ((...,https://www.reddit.com/r/wallstreetbets/commen...,6,4,628rand,1.678915e+09
8,8,AAPL,wallstreetbets,11t61ji,Top changes to open interest calls and puts (T...,https://www.reddit.com/r/wallstreetbets/commen...,5,2,628rand,1.679000e+09
9,9,AAPL,wallstreetbets,11qnf11,Stock Market today 3/13/23,https://i.redd.it/mnyqkk45fmna1.jpg,48,15,cachurch2,1.678745e+09
